In [83]:
import requests as rq
import json
import datetime

In [80]:
# Define some shared variables

# Elastic Search Host
esHost = "http://es:9200"

# Headers
headers = {'Content-Type': 'application/json'}

# Reference layers
usStates = "https://github.com/nawrs/nawrs/raw/GeoJSON/JSON/ReferenceLayers/us_state_BS_5hundthDD.geojson"
tribalLands = "https://raw.githubusercontent.com/nawrs/nawrs/GeoJSON/JSON/ReferenceLayers/tl_2016_us_aiannh_Subset_.001_.geojson"
watersheds = "https://github.com/nawrs/nawrs/raw/GeoJSON/JSON/ReferenceLayers/wbdhu2_PR_1hundthDD.geojson"
documents = ""

# Index Names and Sources
indexInfo = [
    {'indexName':"watersheds", 'dataSource':watersheds, 'typename':"geojson"},
    {'indexName':"triballands", 'dataSource':tribalLands, 'typename':"geojson"},
    {'indexName':"usstates", 'dataSource':usStates, 'typename':"geojson"},
    {'indexName':"nawrs", 'dataSource':documents, 'typeName':"item" }
]

In [81]:
# Get the mappings for the geo indices

for index in indexInfo:
    requestURL = esHost+"/"+index['indexName']+"/_mapping/geojson?pretty"
    print()
    print("Index mapping for: "+requestURL)
    r = rq.get(requestURL)
    print("\tResult: "+str(r.status_code))
    print(json.dumps(r.json(), sort_keys=True, indent=4))



Index mapping for: http://es:9200/watersheds/_mapping/geojson?pretty
	Result: 200
{
    "watersheds": {
        "mappings": {
            "geojson": {
                "properties": {
                    "crs": {
                        "properties": {
                            "properties": {
                                "properties": {
                                    "name": {
                                        "fields": {
                                            "keyword": {
                                                "ignore_above": 256,
                                                "type": "keyword"
                                            }
                                        },
                                        "type": "text"
                                    }
                                }
                            },
                            "type": {
                                "fields": {
                                    

In [51]:
# Get the list of indices

requestURL = esHost+"/_all?pretty"
r=rq.get(requestURL)
print(r.text)

{
  "usstates" : {
    "aliases" : { },
    "mappings" : {
      "geojson" : {
        "properties" : {
          "crs" : {
            "properties" : {
              "properties" : {
                "properties" : {
                  "name" : {
                    "type" : "text",
                    "fields" : {
                      "keyword" : {
                        "type" : "keyword",
                        "ignore_above" : 256
                      }
                    }
                  }
                }
              },
              "type" : {
                "type" : "text",
                "fields" : {
                  "keyword" : {
                    "type" : "keyword",
                    "ignore_above" : 256
                  }
                }
              }
            }
          },
          "features" : {
            "properties" : {
              "geometry" : {
                "type" : "geo_shape",
                "precision" : "100.0m",
                

In [52]:
# Query and related indexed geo-shape search

index = "watersheds"
indexType = "geojson"
searchString = "Pacific"
fieldList = ["NAME"]

requestURL = esHost+"/"+index+"/"+indexType+"/_search"

print()
print(requestURL)

query = {
    "_source": "features.properties.NAME",
    "query"   : {
        "query_string": {
            "query"     : searchString
        }
    }
}

queryJSON = json.dumps(query)

print("\nQuery:")
print(queryJSON)

r=rq.get(requestURL, data=queryJSON, headers=headers)
hits = r.json()['hits']['hits']

#print(hits)

ids = []
for hit in hits:
    print('\nReturned Item:')
    print(hit)
    ids.append(hit['_id'])
print('\nItem IDs for index spatial searching:')
print(ids)



http://es:9200/watersheds/geojson/_search

Query:
{"_source": "features.properties.NAME", "query": {"query_string": {"query": "Pacific"}}}

Returned Item:
{'_source': {'features': {'properties': {'NAME': 'South Pacific Region'}}}, '_type': 'geojson', '_score': 1.4612197, '_id': '9', '_index': 'watersheds'}

Returned Item:
{'_source': {'features': {'properties': {'NAME': 'Pacific Northwest Region'}}}, '_type': 'geojson', '_score': 1.34949, '_id': '4', '_index': 'watersheds'}

Item IDs for index spatial searching:
['9', '4']


In [55]:
# Sequential Search with keyword followed by geometry based search based on keyword results

#startTime = datetime.

# Documents to be searched
docIndex = "nawrs"
docIndexType = "item"
docSearchString = "navajo"
docReturnFields = ["polygon.features.properties.NAME"]
docGeometryField = "polygon.features.geometry"

# Geometries to be searched using the document geometries
refIndex = "watersheds"
refIndexType = "geojson"
refReturnField = ["features.properties.NAME"]

#################################
## search for matching documents
requestURL = esHost+"/"+docIndex+"/"+docIndexType+"/_search"

print('\nRequest URL:')
print(requestURL)

query = {
    "_source": docReturnFields,
    "query"   : {
        "query_string": {
            "query"     : docSearchString
        }
    }
}

queryJSON = json.dumps(query)

print("\nQuery:")
print(queryJSON)

r=rq.get(requestURL, data=queryJSON, headers=headers)
hits = r.json()['hits']['hits']

#print(hits)

docIDs = []
for hit in hits:
    print('\nReturned Item:')
    print(hit)
    docIDs.append(hit['_id'])
print('\nItem IDs for index spatial searching:')
print(docIDs)


#########################################################################################
## search for reference geometries (returning document IDs) using IDs of found documents

geoQueryElements = []
for docID in docIDs:
    geoQueryElement = {
        "geo_shape": {
            "features.geometry": {
                "indexed_shape": {
                    "id": docID,
                    "type": docIndexType,
                    "index": docIndex,
                    "path": docGeometryField
                }
            }
        }
    }
    geoQueryElements.append(geoQueryElement)
    
spatialQuery = {
    "_source": refReturnField,
    "query": {
        "bool": {
            "should": geoQueryElements            
        }
    }
}

queryJSON = json.dumps(spatialQuery)

print("\nQuery:")
print(queryJSON)

requestURL = esHost+"/"+refIndex+"/"+refIndexType+"/_search"
r=rq.get(requestURL, data=queryJSON, headers=headers)
hits = r.json()['hits']['hits']
#print(hits)

refIDs = []
for hit in hits:
    print('\nReturned Item:')
    print(hit)
    refIDs.append(hit['_id'])
print('\nItem IDs for reference features ('+refIndex+') that match spatial search:')
print(refIDs)



Request URL:
http://es:9200/nawrs/item/_search

Query:
{"_source": ["polygon.features.properties.NAME"], "query": {"query_string": {"query": "navajo"}}}

Returned Item:
{'_source': {'polygon': {'features': [{'properties': {'NAME': 'Colorado'}}]}}, '_type': 'item', '_score': 2.6894112, '_id': 'AVsMle9LQe97V9ADs9ze', '_index': 'nawrs'}

Returned Item:
{'_source': {'polygon': {'features': [{'properties': {'NAME': 'Colorado'}}]}}, '_type': 'item', '_score': 2.2591238, '_id': 'AVsMlLZ7Qe97V9ADs9zb', '_index': 'nawrs'}

Item IDs for index spatial searching:
['AVsMle9LQe97V9ADs9ze', 'AVsMlLZ7Qe97V9ADs9zb']

Query:
{"_source": ["features.properties.NAME"], "query": {"bool": {"should": [{"geo_shape": {"features.geometry": {"indexed_shape": {"index": "nawrs", "type": "item", "id": "AVsMle9LQe97V9ADs9ze", "path": "polygon.features.geometry"}}}}, {"geo_shape": {"features.geometry": {"indexed_shape": {"index": "nawrs", "type": "item", "id": "AVsMlLZ7Qe97V9ADs9zb", "path": "polygon.features.geometr

In [97]:
startTime = datetime.datetime.now()

queryString = """{
    "_source": [
        "features.properties.NAME"
    ],
	"query": {
		"bool": {
			"must": {
				"match_all": {}
			},
			"filter": {
				"bool": {
					"should": [
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMhmYkQe97V9ADs9zC",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMjS0CQe97V9ADs9zN",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMjxRIQe97V9ADs9zO",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMkRf6Qe97V9ADs9zT",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMiYZfQe97V9ADs9zH",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMiP83Qe97V9ADs9zG",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMgOeZQe97V9ADs9y6",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVkeXYiEbTu-_lwLAcFj",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMTcPHQe97V9ADs9yk",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVkeWDYGbTu-_lwLAcFb",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMnF1aQe97V9ADs9zq",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMoazJQe97V9ADs9zw",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMle9LQe97V9ADs9ze",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMkv5ZQe97V9ADs9zX",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMpYVkQe97V9ADs9z4",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMl2ilQe97V9ADs9zg",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMovH0Qe97V9ADs9zz",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMiuyeQe97V9ADs9zJ",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMlE05Qe97V9ADs9za",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMkKjUQe97V9ADs9zR",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMk1_2Qe97V9ADs9zY",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMk9T2Qe97V9ADs9zZ",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMfiv0Qe97V9ADs9y2",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMfzXIQe97V9ADs9y4",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMpSSnQe97V9ADs9z3",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMhxGJQe97V9ADs9zD",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMfMeEQe97V9ADs9yz",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMhK3RQe97V9ADs9y_",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMgkrFQe97V9ADs9y8",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMefM-Qe97V9ADs9yw",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMevvIQe97V9ADs9yx",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVkeVsskbTu-_lwLAcFZ",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMgDL3Qe97V9ADs9y5",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMfD2HQe97V9ADs9yy",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMo3DAQe97V9ADs9z0",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMmQ2tQe97V9ADs9zk",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMnOcKQe97V9ADs9zr",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMjDyoQe97V9ADs9zL",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMnXIbQe97V9ADs9zs",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMmC7XQe97V9ADs9zi",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVkeXjLLbTu-_lwLAcFl",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVkeXQ2KbTu-_lwLAcFi",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMeTfdQe97V9ADs9yv",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVkeWfqYbTu-_lwLAcFe",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMPSeQQe97V9ADs9yi",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMZnWoQe97V9ADs9ym",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVkeWwG_bTu-_lwLAcFf",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVkeV7ADbTu-_lwLAcFa",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMkcWJQe97V9ADs9zV",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMiJLCQe97V9ADs9zF",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMoTP-Qe97V9ADs9zv",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMh56eQe97V9ADs9zE",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMjLjdQe97V9ADs9zM",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMlLZ7Qe97V9ADs9zb",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMlYWbQe97V9ADs9zd",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMlR4iQe97V9ADs9zc",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMg9wDQe97V9ADs9y-",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMpKBXQe97V9ADs9z2",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMknuTQe97V9ADs9zW",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMhUHaQe97V9ADs9zA",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMltEtQe97V9ADs9zf",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMl8nuQe97V9ADs9zh",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMkCojQe97V9ADs9zQ",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMfUbzQe97V9ADs9y0",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVkeWQA0bTu-_lwLAcFc",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVkeWYc3bTu-_lwLAcFd",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMgXxvQe97V9ADs9y7",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMfp6MQe97V9ADs9y3",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMnePLQe97V9ADs9zt",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMm-i8Qe97V9ADs9zo",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMo_2jQe97V9ADs9z1",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMmJlAQe97V9ADs9zj",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						},
						{
							"geo_shape": {
								"features.geometry": {
									"indexed_shape": {
										"id": "AVsMmmNxQe97V9ADs9zm",
										"index": "nawrs",
										"path": "polygon.features.geometry",
										"type": "item"
									}
								}
							}
						}
					]
				}
			}
		}
	}
}

"""

queryJSON = json.loads(queryString)
requestURL = "http://www.nawrs.net/es/watersheds%2Cusstates%2Ctriballands/geojson/_search?"
#print(json.dumps(queryJSON))

r=rq.get(requestURL, data=queryString, headers=headers)
hits = r.json()

endTime = datetime.datetime.now()

print(json.dumps(hits))

print("\nElapsed time: "+str(endTime-startTime)+"\n")




{"hits": {"hits": [{"_source": {"features": {"properties": {"NAME": "Fort Peck"}}}, "_type": "geojson", "_score": 1.0, "_id": "190", "_index": "triballands"}, {"_source": {"features": {"properties": {"NAME": "Fort Hall"}}}, "_type": "geojson", "_score": 1.0, "_id": "208", "_index": "triballands"}, {"_source": {"features": {"properties": {"NAME": "Tesuque"}}}, "_type": "geojson", "_score": 1.0, "_id": "227", "_index": "triballands"}, {"_source": {"features": {"properties": {"NAME": "Uintah and Ouray"}}}, "_type": "geojson", "_score": 1.0, "_id": "253", "_index": "triballands"}, {"_source": {"features": {"properties": {"NAME": "Miccosukee"}}}, "_type": "geojson", "_score": 1.0, "_id": "48", "_index": "triballands"}, {"_source": {"features": {"properties": {"NAME": "Warm Springs"}}}, "_type": "geojson", "_score": 1.0, "_id": "60", "_index": "triballands"}, {"_source": {"features": {"properties": {"NAME": "Yavapai-Prescott"}}}, "_type": "geojson", "_score": 1.0, "_id": "126", "_index": "tr